**This notebook presents code associated with Figure 3-Supplement of the following paper.**  
>Kiichi Watanabe, Hui Chiu, David J. Anderson. "Whole brain in situ mapping of neuronal activation in Drosophila during social behaviors and optogenetic stimulation" eLife (2024) (https://doi.org/10.7554/eLife.92380)  
  
**The data**  
The data set (.csv) contains pooled fluorescent intensity data from individual neurons from multiple individual flies.
The jGCaMP7b signal was recorded from 0273-GAL4 positive neurons with optogenetic activation of P1 neurons at 1Hz. ROIs were defined manually, and fluorescence intensity was measured using Fiji/ImageJ. Please also refer to the Materials and Methods section of the paper.

In [ ]:
# Import libraries
%matplotlib inline
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tslearn.preprocessing import TimeSeriesScalerMeanVariance
from sklearn.decomposition import PCA
from scipy import stats
import seaborn as sns

**Create a DataFrame for analysis.**  
The response to 3 repetitive photo-stimulation was recorded. 
The baseline fluorescence signal (F) was calculated by averaging the jGCaMP7b signal 15 sec before the photo-stimulation.  
After calculating dF/F, averaged three responses to create a DataFrame (df_mean).  

**Identification of the neurons that respond to the P1 stimulation.**  
By comparing dF/F before and after the photo-stimulation, we identified "Responsive" neurons and added 'Cell_Identity' to the DataFrame.  
'Cell_Identity'  
0:responsive  
1:non-responsive).

In [ ]:
# Load the data, calculate dF/F0, and classify the neurons
df1 = pd.read_csv('specify the path to .csv file') # define the path to the data

def calculate_dff(df, start, end, baseline_start, baseline_end):
    """
    calculate the dF/F0 from a dataset

    Parameters:
    df (DataFrame): the DataFrame containing the data
    start (int): the starting index for the data set
    end (int): the ending index for the data set
    baseline_start (int): the starting index for the baseline (F0) calculation
    baseline_end (int): the ending index for the baseline (F0) calculation

    Returns:
    DataFrame: the dF/F0 normalized data.
    """
    df_stim = df.iloc[start:end, :]
    baseline_mean = df_stim.iloc[baseline_start:baseline_end, :].mean()
    df_dff = (df_stim - baseline_mean) / baseline_mean # calculate dF/F0
    return df_dff.reset_index (drop=True)

# Calculate dF/F0 for each stimulation (in this case, 3 repetitive stimulation)
df_FA = calculate_dff(df1, 0, 90, 15, 30)      # First stimulation
df_FB = calculate_dff(df1, 60, 150, 15, 30)    # Second stimulation
df_FC = calculate_dff(df1, 120, 210, 15, 30) # Third stimulation

# Average 3 responses from the same neuron
df_mean = (df_FA + df_FB + df_FC) / 3
df_mean = df_mean.T  # Transpose for further analysis

# Initialize Identity DataFrame to classify cells
Identity = pd.DataFrame(index=df_mean.index, columns=['Cell_Identity'])

# Total number of rows (neurons) in df_mean
ntotalcells = df_mean.shape[0]

# Iterate through each row (neuron) to classify based on Mann-Whitney U test
for ii in range(ntotalcells):
    A = df_mean.iloc[ii, 15:30]  # Data before stimulation (pre-stimulation)
    B = df_mean.iloc[ii, 30:60]  # Data during stimulation
    
    # Mann-Whitney U test
    C = stats.mannwhitneyu(A, B, alternative='less')
    
    mA = A.mean()  # Mean of pre-stimulation period
    mB = B.mean()  # Mean of stimulation period
    P = C.pvalue   # Extract the p-value
    
    # Classify based on p-value and comparison of means
    if P < 0.01 and mA < mB:
        Identity.loc[df_mean.index[ii], 'Cell_Identity'] = 0  # Responsive
    else:
        Identity.loc[df_mean.index[ii], 'Cell_Identity'] = 1  # Non-responsive

# Add the classification to df_mean
df_mean['Cell_Identity'] = Identity['Cell_Identity']

# Display the first few rows of the updated DataFrame
print(df_mean.head())

# Count the number of responsive neurons (where 'Cell_Identity' == 0)
count_zeros = (df_mean['Cell_Identity'] == 0).sum()
print(f"Number of responsive neurons (0s in 'Cell_Identity'): {count_zeros}")

**Creating heatmaps for Responsive and Non-responsive neuronal populations (Figure S3I)**  
We performed PCA on the activities of the entire population (df_mean) and used the PC1 value to order neurons.

In [ ]:
# Separate the time point data and 'Cell_Identity' column
df_time_points = df_mean.drop('Cell_Identity', axis=1)  # Exclude 'Cell_Identity' column
# df_identity = df_mean['Cell_Identity']  # Keep 'Cell_Identity' column separately

# Normalize only the time point columns (the dF/F0 values)
T_Norm = TimeSeriesScalerMeanVariance(mu=0.0, std=1.0).fit_transform(df_time_points.values)

# Convert the 3D array to 2D by squeezing the last dimension
T_Norm_2D = T_Norm.squeeze()  # Remove the singleton dimension (shape becomes [175, 90])

# Step 4: Convert the normalized NumPy array back to a DataFrame, restoring the original column names
df_mean_Norm = pd.DataFrame(T_Norm_2D, columns=df_time_points.columns, index=df_time_points.index)

# Step 6: Preview the final normalized DataFrame
print(df_mean_Norm.head())

In [ ]:
# To visualize the responses of neurons, create a heatmap
# Use PCA and PC1 values to order neurons

from sklearn.decomposition import PCA

# Perform PCA on the normalized data
pca = PCA(n_components=10)
pca.fit(df_mean_Norm)

# Transform data into principal components
feature = pca.transform(df_mean_Norm)

# Create a DataFrame for PCA results
loadings = pd.DataFrame(feature, columns=["PC{}".format(x + 1) for x in range(10)], index=df_mean_Norm.index)

# Preview the loadings
print(loadings.head())

# Add PC1 values to the original DataFrame (df_mean_New) for further sorting
df_mean['PC1'] = loadings['PC1']
sort_df_mean = df_mean.sort_values('PC1')

# Separate neurons by their response classification
df_mean_P = sort_df_mean[sort_df_mean['Cell_Identity'] == 0]  # Positive response (responsive)
df_mean_N = sort_df_mean[sort_df_mean['Cell_Identity'] == 1]  # No response

# Check if DataFrames are sorted by PC1 value
print('df_mean_New_P')
print(df_mean_P.head())
print('df_mean_New_N')
print(df_mean_N.head())

# Drop 'Cell_Identity' and 'PC1' columns for plotting
plot_df_P = df_mean_P.drop(['Cell_Identity', 'PC1'], axis=1)
plot_df_N = df_mean_N.drop(['Cell_Identity', 'PC1'], axis=1)

# Select data from -15 to +30 time points (corresponding to stimulation)
plot_df_P2 = plot_df_P.iloc[:, 15:90]
plot_df_N2 = plot_df_N.iloc[:, 15:90]

In [ ]:
# Create Heatmaps for Responsive and Non-responsive Neurons
# Check info for the datasets and double-check the number of neurons from each population
print(plot_df_P2.info())
print(plot_df_N2.info())

# Plot heatmap of dF/F for positive responses (responsive neurons)
%matplotlib inline
plt.figure()
sns.heatmap(plot_df_P2, vmin=-0.0, vmax=0.5, cmap='jet')
plt.title('Responsive Neurons (Positive Response)')
#plt.savefig('0273_dF_F_Positive.pdf', dpi=600)  # Define the name
plt.show()

# Plot heatmap of dF/F for no response neurons
plt.figure()
sns.heatmap(plot_df_N2, vmin=-0.0, vmax=0.5, cmap='jet')
plt.title('Non-responsive Neurons (No Response)')
#plt.savefig('0273_dF_F_NoResponse.pdf', dpi=600)  # Define the name
plt.show()

**Fraction of 0273 PAM neurons reponsive to P1 stimulation (Figure S3J).**  
Now, we characterize 0271 PAM neurons based on the responsiveness to the P1 stimulation.

In [ ]:
# Analysis of fold change during the stimulation for further characterization of the responsive neurons
# Calculate the average fold change during the stimulation period (15:45)
df_4foldchange = plot_df_P2.iloc[:, 15:45]
A = df_4foldchange.mean(axis=1)

# Create a DataFrame for the mean fold changes during stimulation
mean_dF_F = pd.DataFrame()
mean_dF_F['mean_dF_F'] = A

# Calculate the number of cells with strong and intermediate responses
strong_cells = mean_dF_F[mean_dF_F['mean_dF_F'] > 0.4]  # dF/F > 0.4
intermediate_cells = mean_dF_F[mean_dF_F['mean_dF_F'] > 0.15]  # dF/F > 0.15 (including strong cells)

# Counts
strong = strong_cells.shape[0]
intermediate = intermediate_cells.shape[0] - strong  # Intermediate cells exclude strong cells
weak = plot_df_P2.shape[0] - (strong + intermediate)  # Remaining positive response cells are weak
no_response = plot_df_N2.shape[0]  # No response cells
total_cells = df_mean.shape[0]  # Total number of cells

# Display counts
print(f"Strong response cells: {strong}")
print(f"Intermediate response cells: {intermediate}")
print(f"Weak response cells: {weak}")
print(f"No response cells: {no_response}")
print(f"Total cells: {total_cells}")

# Normalize counts for the bar plot
bar_y = [strong/total_cells, intermediate/total_cells, weak/total_cells, no_response/total_cells]
bar_x = ['>0.4 (Strong)', '>0.15 (Intermediate)', '<0.15 (Weak)', 'No Response']

# Plot the bar graph
plt.style.use('default')
fig, ax = plt.subplots()

# Bar plot showing the percentage of neurons by response type
plt.bar(bar_x, bar_y, width=0.5, color='gray', edgecolor='black')

# Add titles and labels
ax.set_title('Ratio of Neurons Responding to P1 Stimulation')
ax.set_xlabel('Response Criteria (dF/F thresholds)')
ax.set_ylabel('Percentage of Neurons')
ax.set_ylim(0, 1.0)
ax.set_yticks([0, 0.25, 0.5, 0.75, 1.0])

#plt.savefig('0273_P1_%responding.pdf')
plt.show()
